In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import dython
import pickle
import joblib
from datetime import datetime
from dateutil import parser
from dython.nominal import associations

In [2]:
cyber = pd.read_csv("../data/cic_ids_2018.csv")
cyber.head()

,Unnamed: 0,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,3389,1665875,8,7,1128,1581.0,661,0,141.00,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,1,53,67765,2,2,94,268.0,47,47,47.00,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,2,0,213190,5,0,0,0.0,0,0,0.00,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,3,41967,86370853,2,0,0,0.0,0,0,0.00,...,20,0.0,0.0,0.0,0.0,86400000.0,0.0,86400000.0,86400000.0,Benign
4,4,80,5113386,4,4,97,231.0,97,0,24.25,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [3]:
cyber.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246730 entries, 0 to 246729
Data columns (total 74 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         246730 non-null  int64  
 1   Dst Port           246730 non-null  int64  
 2   Flow Duration      246730 non-null  int64  
 3   Tot Fwd Pkts       246730 non-null  int64  
 4   Tot Bwd Pkts       246730 non-null  int64  
 5   TotLen Fwd Pkts    246730 non-null  int64  
 6   TotLen Bwd Pkts    246730 non-null  float64
 7   Fwd Pkt Len Max    246730 non-null  int64  
 8   Fwd Pkt Len Min    246730 non-null  int64  
 9   Fwd Pkt Len Mean   246730 non-null  float64
 10  Fwd Pkt Len Std    246730 non-null  float64
 11  Bwd Pkt Len Max    246730 non-null  int64  
 12  Bwd Pkt Len Min    246730 non-null  int64  
 13  Bwd Pkt Len Mean   246730 non-null  float64
 14  Bwd Pkt Len Std    246730 non-null  float64
 15  Flow IAT Mean      246730 non-null  float64
 16  Fl

In [4]:
cyber.isna().sum()


Unnamed: 0       0
Dst Port         0
Flow Duration    0
Tot Fwd Pkts     0
Tot Bwd Pkts     0
                ..
Idle Mean        0
Idle Std         0
Idle Max         0
Idle Min         0
Label            0
Length: 74, dtype: int64

In [5]:
start = []
for col in cyber.columns:
        if col[:7] == "Bwd IAT": 
                start.append(col)
print(start)


['Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min']


In [6]:
cyber = cyber.replace(0, np.nan)

In [7]:
cyber.dropna(how = "all", axis =1,inplace = True )
cyber.fillna(0, inplace = True)
cyber.head()

,Unnamed: 0,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.0,3389.0,1665875.0,8,7.0,1128.0,1581.0,661.0,0.0,141.00,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,1.0,53.0,67765.0,2,2.0,94.0,268.0,47.0,47.0,47.00,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,2.0,0.0,213190.0,5,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,3.0,41967.0,86370853.0,2,0.0,0.0,0.0,0.0,0.0,0.00,...,20.0,0.0,0.0,0.0,0.0,86400000.0,0.0,86400000.0,86400000.0,Benign
4,4.0,80.0,5113386.0,4,4.0,97.0,231.0,97.0,0.0,24.25,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [8]:
cyber.dropna(how = "all", axis =0,inplace = True )
cyber.shape

(246730, 66)

In [9]:
X= cyber.drop(['Label',"Unnamed: 0"], axis =1)
y =cyber["Label"]

In [10]:
from sklearn.model_selection import train_test_split
np.random.seed(42)
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset
data = pd.read_csv('../data/cic_ids_2018.csv')

# Handle missing values by dropping rows with any None or NaN values
data = data.dropna()

# Encode categorical labels
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

# Separate features and labels
X = data.drop('Label', axis=1)
y = data['Label']

# Drop columns with all zero values
X = X.loc[:, (X != 0).any(axis=0)]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Print the columns that start with "Bwd IAT"
start = []
for col in X.columns:
    if col[:7] == "Bwd IAT":
        start.append(col)
print(start)

# Handle missing values again (if any)
X.dropna(how='all', axis=1, inplace=True)
X.fillna(0, inplace=True)
X.dropna(how='all', axis=0, inplace=True)

# Convert X_scaled back to DataFrame for further processing
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

['Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min']


In [14]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=100))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(X.shape[1], activation='tanh'))
    return model

# Define the discriminator model (critic in WGAN)
def build_critic():
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=X.shape[1]))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1))
    return model

# Build the generator and critic
generator = build_generator()
critic = build_critic()

# Define the WGAN class
class WGAN(tf.keras.Model):
    def __init__(self, generator, critic, gp_weight=10.0):
        super(WGAN, self).__init__()
        self.generator = generator
        self.critic = critic
        self.gp_weight = gp_weight

    def compile(self, g_optimizer, c_optimizer, g_loss_fn, c_loss_fn):
        super(WGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.c_optimizer = c_optimizer
        self.g_loss_fn = g_loss_fn
        self.c_loss_fn = c_loss_fn

    def gradient_penalty(self, batch_size, real_data, fake_data):
        alpha = tf.random.normal([batch_size, 1], 0.0, 1.0)
        diff = fake_data - real_data
        interpolated = real_data + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.critic(interpolated, training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_data):
        batch_size = tf.shape(real_data)[0]
        for _ in range(5):
            noise = tf.random.normal([batch_size, 100])
            with tf.GradientTape() as c_tape:
                fake_data = self.generator(noise, training=True)
                real_output = self.critic(real_data, training=True)
                fake_output = self.critic(fake_data, training=True)
                c_loss = self.c_loss_fn(real_output, fake_output)
                gp = self.gradient_penalty(batch_size, real_data, fake_data)
                c_loss += gp * self.gp_weight

            c_grads = c_tape.gradient(c_loss, self.critic.trainable_variables)
            self.c_optimizer.apply_gradients(zip(c_grads, self.critic.trainable_variables))

        noise = tf.random.normal([batch_size, 100])
        with tf.GradientTape() as g_tape:
            fake_data = self.generator(noise, training=True)
            fake_output = self.critic(fake_data, training=True)
            g_loss = self.g_loss_fn(fake_output)

        g_grads = g_tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {"c_loss": c_loss, "g_loss": g_loss}

# Define the loss functions
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

def critic_loss(real_output, fake_output):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

# Instantiate the WGAN model
wgan = WGAN(generator, critic)

# Compile the WGAN model
wgan.compile(
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.9),
    c_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.9),
    g_loss_fn=generator_loss,
    c_loss_fn=critic_loss
)

/Users/sanjanathyady/Desktop/project-2/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
# Training parameters
epochs = 10000
batch_size = 64
save_interval = 1000  # Save the model every 1000 epochs

# Training loop
for epoch in range(epochs):
    idx = np.random.randint(0, X_scaled.shape[0], batch_size)
    real_data = X_scaled[idx]
    wgan.train_step(real_data)

    if epoch % save_interval == 0:
        # Save the generator and critic models
        generator.save(f'generator_epoch_{epoch}.h5')
        critic.save(f'critic_epoch_{epoch}.h5')
        print(f"Epoch: {epoch} - Models saved")

    if epoch % 1000 == 0:
        print(f"Epoch: {epoch}")

Epoch: 0
Epoch: 1000
Epoch: 2000
Epoch: 3000
Epoch: 4000


KeyboardInterrupt: 

In [ ]:
# Generate new attack samples
noise = np.random.normal(0, 1, (1000, 100))
generated_data = generator.predict(noise)

# Inverse transform to original scale
generated_data = scaler.inverse_transform(generated_data)

In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd
import joblib

# Load the saved generator model
generator = tf.keras.models.load_model('generator_epoch_600.h5')

# Load the scaler

# Generate new attack samples
noise = np.random.normal(0, 1, (1000, 100))
generated_data = generator.predict(noise)

# Inverse transform to original scale
generated_data = scaler.inverse_transform(generated_data)

# Convert the generated data to a DataFrame
generated_df = pd.DataFrame(generated_data)

# Save the DataFrame to a CSV file
generated_df.to_csv('generated_attack_data.csv', index=False)

# Print the generated data
print(generated_data)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[ 1.9448747e+05 -3.3237442e+02 -2.2459952e+07 ... -1.1344434e+05
  -1.0402663e+06 -1.1324042e+06]
 [ 5.4202789e+04 -2.4433950e+03 -2.6691850e+08 ...  1.0283628e+05
  -5.3337431e+05  2.4223880e+05]
 [ 5.2139688e+04  2.0000260e+04 -1.1424272e+08 ...  2.1217062e+05
   8.8915994e+05  1.2470579e+05]
 ...
 [ 1.4361583e+05 -1.4197972e+03 -6.0625252e+07 ...  7.3155125e+04
  -1.0520378e+06 -1.2970102e+06]
 [ 5.2146945e+04 -4.6831255e+03 -1.5843888e+08 ...  1.7122219e+05
  -1.4363402e+06  3.5944553e+05]
 [ 1.3784159e+05  1.9480490e+03  9.2544368e+07 ...  4.2116504e+04
  -3.1893928e+05 -4.5878822e+05]]
